### Vamos a dejar distintos tipos de tests donde tratamos de abusar de los rankings

In [2]:
from subprocess import *
import os
import pandas as pd

#---------------CONFIGURACION---------------#
metodos = {
    "cmm": "0",
    "wp": "1",
    "elo": "2"
}
GLOBAL_PATH = os.getcwd()
executable = GLOBAL_PATH+'/../tp' if os.name == 'posix' else GLOBAL_PATH+'/../tp.exe' #Para que funcione en todos los SO.
outputPath = GLOBAL_PATH+'/tmp/comparacion_ranking_jupyter.out' #Archivo donde nos va a devolver el resultado el programa de C++
inputPath  = GLOBAL_PATH+'/tmp/comparacion_ranking_jupyter.in' #Archivo donde vamos a escribir la entrada para que pueda leer el programa de C++

def call(*popenargs, timeout=None, **kwargs):
    with Popen(*popenargs, **kwargs) as p:
        try:
            return p.wait(timeout=timeout)
        except:  
            p.kill()
            raise
            
def runTP(data, nombresEquipos, metodo):
    
    text_file = open(inputPath, "w")
    text_file.write(data)
    text_file.close()
    call([executable, inputPath, outputPath, metodo])
    with open(outputPath, 'r') as factual:
      results = [float(x.strip()) for x in factual.readlines() if len(x.strip()) > 0]
    
    return [x for _,x in sorted(zip(results,nombresEquipos), reverse=True)] #Devolvemos los equipos ordenados por el ranking
    
def runTPWithAllMethods(data, nombresEquipos):
    result = pd.DataFrame()
    for x in dict.keys(metodos):
        result.insert(len(result.columns),x,runTP(data, nombresEquipos, metodos[x]))
        
    return result


### 1 - Ganarle al equipo del mejor rating (al principio)

In [3]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C','D','E','F']
data = """6 5
1 1 0 6 1
1 1 1 2 0
1 1 1 3 0
1 1 1 4 0
1 1 1 5 0"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   F  F   A
1   A  A   F
2   E  E   E
3   D  D   D
4   C  C   C
5   B  B   B


### 1b - Ganarle al equipo del mejor rating (al final)

In [4]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C','D','E','F']
data = """6 5
1 1 1 2 0
1 1 1 3 0
1 1 1 4 0
1 1 1 5 0
1 1 0 6 1"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   F  F   F
1   A  A   A
2   E  E   E
3   D  D   D
4   C  C   C
5   B  B   B


### 2 - Ganarle muchas veces a alguien vs ganarle una sola vez

In [5]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C']
data = """3 9
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 1 1 2 0
1 3 1 2 0"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   A  C   A
1   C  A   C
2   B  B   B


### 3 - Ganar todos los partidos menos uno vs ganar un solo partido

In [6]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C','D','E','F']
data = """6 11
1 1 1 2 0
1 1 1 3 0
1 1 0 4 1
1 1 1 5 0
1 2 1 3 0
1 2 1 4 0
1 2 1 5 0
1 3 1 4 0
1 3 1 5 0
1 4 1 5 0
1 6 1 5 0"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   B  F   B
1   A  B   A
2   F  A   F
3   D  D   C
4   C  C   D
5   E  E   E


### 4 - Equipo que mejora con el tiempo

In [7]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C','D','E','F']
data = """6 10
1 1 0 2 1
1 1 0 3 1
1 1 0 4 1
1 1 0 5 1
1 1 0 6 1
1 1 1 2 0
1 1 1 3 0
1 1 1 4 0
1 1 1 5 0
1 1 1 6 0"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   F  F   A
1   E  E   F
2   D  D   E
3   C  C   D
4   B  B   C
5   A  A   B


### 5 - Ganarle a los mejores vs ganarle a los peores

In [10]:
#fecha, equipo1, goles1, equipo2, goles2
nombresEquipos = ['A','B','C','D','E','F','G','H','I']
data = """9 9
1 1 1 3 0
1 1 1 4 0
1 1 1 5 0
1 3 1 6 0
1 4 1 7 0
1 5 1 8 0
1 2 1 6 0
1 2 1 7 0
1 2 1 8 0"""

print(runTPWithAllMethods(data, nombresEquipos))

  cmm wp elo
0   A  B   A
1   B  A   B
2   E  E   E
3   D  D   D
4   C  C   C
5   I  I   I
6   H  H   H
7   G  G   G
8   F  F   F


### 6 - 